In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#### Steps
- Read Data
- Tokenize using spacy

In [41]:
import sys
import os
LOCAL_FAST_AI = '/Users/krishnakalyan3/Educational/FastAI/fastai/'
AWS_FAST_AI = '/home/ubuntu/fastai'
sys.path.append(LOCAL_FAST_AI)

In [13]:
from fastai.imports import *
from fastai.transforms import * 
from fastai.conv_learner import * 
from fastai.model import *
from fastai.dataset import * 
from fastai.sgdr import *
from fastai.plots import *
from fastai.learner import *
import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling
from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

In [28]:
PATH = '/Users/krishnakalyan3/Educational/FastAI/code/data/aclImdb/'

In [29]:
!ls {PATH}

README     imdb.vocab imdbEr.txt test       train


In [30]:
TRAIN = PATH + 'train/all/'
TEST = PATH + 'test/all/'

In [31]:
TRAIN_FILES = !ls {TRAIN}
TRAIN_FILES[0:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt',
 '10002_0.txt']

In [32]:
review = !cat {TRAIN}{TRAIN_FILES[0]}
review[0]

'I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?'

In [33]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [34]:
bs=64; bptt=70
FILES = dict(train=TRAIN, validation=TEST, test=TEST)

In [35]:
PATH

'/Users/krishnakalyan3/Educational/FastAI/code/data/aclImdb/'

In [36]:
FILES

{'test': '/Users/krishnakalyan3/Educational/FastAI/code/data/aclImdb/test/all/',
 'train': '/Users/krishnakalyan3/Educational/FastAI/code/data/aclImdb/train/all/',
 'validation': '/Users/krishnakalyan3/Educational/FastAI/code/data/aclImdb/test/all/'}

In [39]:
#md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [40]:
print('hi')

hi
